This Notebook aims to mimic "Insurace Claim" Scenario with multi agent handling different scenario

Note that this notebook is variation of Farzad Sunavala's blog post on multi agent claim processing

https://farzzy.hashnode.dev/streamlining-insurance-claim-analysis-with-semantic-kernels-multi-agent-orchestration


In [2]:
import asyncio
import os
from pathlib import Path
from datetime import datetime, timedelta
from dotenv import load_dotenv

import sys

from semantic_kernel.kernel import Kernel
from semantic_kernel.agents.open_ai.azure_assistant_agent import AzureAssistantAgent
from semantic_kernel.agents import AgentGroupChat, ChatCompletionAgent

from semantic_kernel.agents.strategies.selection.kernel_function_selection_strategy import KernelFunctionSelectionStrategy
from semantic_kernel.agents.strategies.termination.kernel_function_termination_strategy import KernelFunctionTerminationStrategy

from semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion import AzureChatCompletion
from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import AzureChatPromptExecutionSettings

from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.contents.chat_message_content import ChatMessageContent

from semantic_kernel.agents.strategies.termination.default_termination_strategy import DefaultTerminationStrategy
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy

from semantic_kernel.functions.kernel_function_from_prompt import KernelFunctionFromPrompt

from semantic_kernel.core_plugins.web_search_engine_plugin import WebSearchEnginePlugin

from semantic_kernel.connectors.search_engine import BingConnector
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior

load_dotenv(override=True)

AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")  
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

BING_API_KEY = os.getenv("BING_SEARCH_SUBSCRIPTION_KEY")


In [3]:
def create_kernel(service_id: str) -> Kernel:
    kernel = Kernel()
    kernel.add_service(
        AzureChatCompletion(
            service_id=service_id,
            deployment_name=AZURE_OPENAI_DEPLOYMENT,
            endpoint=AZURE_OPENAI_ENDPOINT,
            api_key=AZURE_OPENAI_API_KEY,
            api_version=AZURE_OPENAI_API_VERSION,
        )
    )
    return kernel


In [8]:
relocation_overseer = await AzureAssistantAgent.create(
    kernel=create_kernel("relocation_overseer"),
    deployment_name=AZURE_OPENAI_DEPLOYMENT,
    endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    name="Relocation_Overseer",
    instructions="""
    You are an Overseer who manages the relocation process for employees.
    You have access to other agents "Flight Searcher" and "Housing Locator" to help you with the process.
    User will provide the relocation detail such as relocation date, destination, and preferences.

    YOUR PROCESS:
    1. First Response:
    - Take the information from the user and parse it into a JSON format such as:
    {
        "relocation_date": "YYYY-MM-DD",
        "destination": "city, country",
        "preferences": ["near office", "close to school"]
    }
    2. You will then wait for both "Flight Searcher" and "Housing Locator" to provide their input.

    3. Final Response: Only after both agents have provided their input, you will make the final recommendation.
    MAKE SURE TO INCLUDE THE URL SOURCE FOR BOTH FLIGHT AND HOUSING OPTIONS.
    - Structure the final recommendation in JSON format:
    {
        "completed": true|false,
        "flight": {...},
        "housing": {...},
        "total_cost": number
    }

    KEY REQUIREMENTS:
    - Wait for both other agents before final summerizaion.
    - Consider the price the most important factor.
    - Preference would be considered if the price is the same.
    """,
)

flight_searcher_kernel = create_kernel("flight_searcher")

connector = BingConnector(api_key=BING_API_KEY)
flight_searcher_kernel.add_plugin(WebSearchEnginePlugin(connector), plugin_name="WebSearch")

settings = flight_searcher_kernel.get_prompt_execution_settings_from_service_id(service_id="flight_searcher")
settings.function_choice_behavior = FunctionChoiceBehavior.Auto()


flight_searcher = ChatCompletionAgent(
    kernel=flight_searcher_kernel,
    name="Flight_Searcher",
    instructions="""
    You are a Flight Searcher who helps find the best flight options.
    You have access to Web Search Plugin to help you find the best flight options.
    Using the information from Relocation Overseer, such as departure location, destination, and date, find the best flight options.
    ONLY RETURN LIST OF FLIGHTS, NO OTHER MESSAGE IS REQUIRED.
    Be sure to include the following information in your response:
    - Flight number
    - Flight options with prices
    - link to source
    - Layover information (if any)
    """,
)

housing_locator_kernel = create_kernel("housing_locator")

connector = BingConnector(api_key=BING_API_KEY)
housing_locator_kernel.add_plugin(WebSearchEnginePlugin(connector), plugin_name="WebSearch")

settings = housing_locator_kernel.get_prompt_execution_settings_from_service_id(service_id="housing_locator")
settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

housing_locator = ChatCompletionAgent(
    kernel=create_kernel("housing_locator"),
    name="Housing_Locator",
    instructions="""
    You are a Housing Locator who helps find the best housing options for the relocation process.
    You have access to Web Search Plugin to help you find the best housing options.
    ONLY RETURN LIST OF HOUSING OPTIONS, NO OTHER MESSAGE IS REQUIRED.
    Consider the user's preferences when providing housing options.
    Be sure to include the following information in your response:
    - Housing options with prices
    - link to source
    """,
)


selection_function = KernelFunctionFromPrompt(
    function_name="selection",
    prompt=f"""
    Determine which participant takes the next turn in a conversation based on the most recent participant.
    State only the name of the participant to take the next turn.
    No participant should take more than one turn in a row.

    Choose only from these participants:
    - Relocation_Overseer
    - Flight_Searcher
    - Housing_Locator

    Always follow these rules when selecting the next participant:
    - After user input, it is Relocation_Overseer's turn.
    - After Relocation_Overseer replies, it is Flight_Searcher's turn.
    - MAKE SURE TO WAIT UNTIL Flight_Searcher PROVIDES LIST OF FLIGHTS before Housing_Locator.
    - After Flight_Searcher replies, it is Housing_Locator's turn.
    - After Both Flight_Searcher and Housing_Locator replies, it is Relocation_Overseer's turn to make the final summerization.
    - when Relocation_Overseer makes the final summerization, with a json "completed": true, the chat is terminated.

    History:
    {{{{$history}}}}
    """,
)

termination_function = KernelFunctionFromPrompt(
    function_name="termination",
    prompt=f"""
        Examine the RESPONSE and determine whether the content has been deemed satisfactory.
        The response is satisfactory if the content is a JSON object with the key "completed" and the value of "true".
        If content is satisfactory, respond with a single word without explanation: "termiante_chat".

        RESPONSE:
        {{{{$history}}}}
    """,
)



try:
    chat = AgentGroupChat(
        agents=[relocation_overseer, flight_searcher, housing_locator],
        selection_strategy=KernelFunctionSelectionStrategy(
            function=selection_function,
            kernel=create_kernel("selection"),
            result_parser=lambda result: str(result.value[0]) if result.value is not None else "Relocation_Overseer",
            agent_variable_name="agents",
            history_variable_name="history",
        ),
        # termination_strategy=DefaultTerminationStrategy(maximum_iterations=4),
        # termination_strategy=ApprovalTerminationStrategy(agents=[claims_analyst], maximum_iterations=10),
        termination_strategy=KernelFunctionTerminationStrategy(
            agents=[relocation_overseer],
            function=termination_function,
            kernel=create_kernel("termination"),
            result_parser=lambda result: "termiante_chat" in str(result.value[0]).lower(),
            history_variable_name="history",
            maximum_iterations=8,
        ),


    )

    input = """
    I am planning to relocate from New York to Microsoft offie at London, UK on 2025-04-01, for 3 months.
    I would like to find a flight and housing options.
    My preferences are to find a flight with a layover less than 2 hours and housing near the office.
    """
    # Add the initial message
    await chat.add_chat_message(
        ChatMessageContent(
            role=AuthorRole.USER, 
            content=input
        )
    )
    
    # print green user icon and the input
    print("\n 🙂 User:")
    print(input)

    current_agent = None
    async for content in chat.invoke():
        print(f"\n👤 {content.name}:")
        print(content.content.strip())
            
# print(f"# IS COMPLETE: {chat.is_complete}")
except Exception as e:
    await chat.reset()


 🙂 User:

    I am planning to relocate from New York to Microsoft offie at London, UK on 2025-04-01, for 3 months.
    I would like to find a flight and housing options.
    My preferences are to find a flight with a layover less than 2 hours and housing near the office.
    

👤 Relocation_Overseer:
Thank you for providing the details of your relocation. Here is the parsed information:

```json
{
    "relocation_date": "2025-04-01",
    "destination": "London, UK",
    "preferences": ["layover less than 2 hours", "housing near the office"]
}
```

I will now reach out to "Flight Searcher" and "Housing Locator" for their input. Please await the final recommendation.

👤 Flight_Searcher:
Searching for the best flight options now.

---

**Flight Options from New York (JFK) to London (LHR) on 2025-04-01:**

1. **Flight Number**: BA178
   - **Airline**: British Airways
   - **Departure**: 8:00 PM (JFK)
   - **Arrival**: 8:10 AM next day (LHR)
   - **Non-stop**: Yes
   - **Price**: $620
   -

Failed to select agent: Agent Failure - Strategy unable to select next agent: The conversation has ended with Relocation_Overseer's final summarization. Therefore, no participant will take the next turn as the chat is terminated.
